In [19]:
import findspark
findspark.init('/spark/spark-3.5.1-bin-hadoop3')
from pyspark import *
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DateType, TimestampType, LongType
from pyspark.sql.types import ArrayType, DoubleType, BooleanType, DecimalType
from pyspark.sql.functions import regexp_extract, split, from_unixtime, col, avg, min, max, desc
from pyspark.sql.functions import grouping, explode, array_contains



spark = SparkSession.builder.appName("movielens").getOrCreate()

# Consultas sobre Movielens

## Schema de Movielens

![Schema](movielens_schema.png)

## ¿Cuál es la distribución de la clasificación de las películas? 

Proceso:

1. Cargar la tabla ratings
1. Cargar la tabla movies
1. Unir ambas tablas
1. Agrupar por calificación
1. Contar las películas


### Cargar las tablas

In [2]:
# Tabla Ratings
ratings_schema  = StructType(fields=[
    StructField("userId",IntegerType(),True), 
    StructField("movieId",IntegerType(),True),
    StructField("rating",DecimalType(precision=2,scale=1),True),
    StructField("timestamp",LongType(),True)
])
ratingsDf = spark.read\
    .option("header", True)\
    .option("dateFormat", "yyyyMMdd")\
    .schema(ratings_schema)\
    .csv("./ml-latest-small/ratings.csv")\
    .withColumn(\
            "date",\
            from_unixtime("timestamp", "yyyyMMdd"))\
                .drop('timestamp')

# Tabla Movies
movies_schema  = StructType(fields=[
    StructField("movieId",IntegerType(),True), 
    StructField("title",StringType(),True),
    StructField("genres",StringType(),True)
])

moviesDf = spark.read\
    .option("header", True)\
    .schema(movies_schema)\
    .csv("./ml-latest-small/movies.csv")

moviesDf = moviesDf.withColumn("genresSplit", split(moviesDf["genres"],"\|"))\
                        .drop('genres').withColumnRenamed("genresSplit","genres")\
                            .withColumn(\
                                "year",\
                                regexp_extract(\
                                           moviesDf["title"],\
                                           "^.+\(([0-9]+)\)$",\
                                           1)\
                                .cast(IntegerType()))\
                            .withColumn(\
                            "title_temp",\
                            regexp_extract(\
                                           moviesDf["title"],\
                                           "^(.+?) \([0-9]+\)$",\
                                           1))\
                            .drop('title')\
                        .withColumnRenamed("title_temp","title")

### Unir Ambas tablas

In [3]:
movie_ratingsDF = ratingsDf.join(moviesDf,on="movieId",how="inner")
movie_ratingsDF.head(3)

[Row(movieId=1, userId=1, rating=Decimal('4.0'), date='20000730', genres=['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy'], year=1995, title='Toy Story'),
 Row(movieId=3, userId=1, rating=Decimal('4.0'), date='20000730', genres=['Comedy', 'Romance'], year=1995, title='Grumpier Old Men'),
 Row(movieId=6, userId=1, rating=Decimal('4.0'), date='20000730', genres=['Action', 'Crime', 'Thriller'], year=1995, title='Heat')]

### Filtros

In [9]:
movie_ratingsDF.filter("movieId = 356").select(["rating","genres"]).head(3)

[Row(rating=Decimal('4.0'), genres=['Comedy', 'Drama', 'Romance', 'War']),
 Row(rating=Decimal('5.0'), genres=['Comedy', 'Drama', 'Romance', 'War']),
 Row(rating=Decimal('5.0'), genres=['Comedy', 'Drama', 'Romance', 'War'])]

In [16]:
movie_ratingsDF.filter(col('genres').getItem(0) == 'Drama').select(["rating","genres"]).head(3)

[Row(rating=Decimal('5.0'), genres=['Drama', 'War']),
 Row(rating=Decimal('5.0'), genres=['Drama', 'Mystery']),
 Row(rating=Decimal('4.0'), genres=['Drama', 'Fantasy', 'Romance'])]

In [21]:
movie_ratingsDF.filter(array_contains(movie_ratingsDF.genres,'Drama')).select(["rating","genres"]).head(10)

[Row(rating=Decimal('4.0'), genres=['Action', 'Drama', 'War']),
 Row(rating=Decimal('5.0'), genres=['Action', 'Drama', 'Romance', 'War']),
 Row(rating=Decimal('4.0'), genres=['Comedy', 'Drama']),
 Row(rating=Decimal('3.0'), genres=['Comedy', 'Crime', 'Drama', 'Thriller']),
 Row(rating=Decimal('4.0'), genres=['Action', 'Crime', 'Drama', 'Thriller']),
 Row(rating=Decimal('4.0'), genres=['Comedy', 'Drama', 'Romance', 'War']),
 Row(rating=Decimal('3.0'), genres=['Comedy', 'Drama']),
 Row(rating=Decimal('5.0'), genres=['Drama', 'War']),
 Row(rating=Decimal('5.0'), genres=['Action', 'Drama', 'Western']),
 Row(rating=Decimal('4.0'), genres=['Adventure', 'Drama', 'Western'])]

# Funciones de Ventana en PySpark

## ¿Qué son las Funciones de Ventana?

Las funciones de ventana en PySpark permiten realizar cálculos avanzados sobre un conjunto específico de filas relacionadas, conocidas como "ventanas", sin la necesidad de agrupar los datos en su totalidad. Estas funciones son muy útiles para realizar análisis complejos y obtener insights detallados de los datos, manteniendo el contexto de cada fila.

### Características Principales de las Funciones de Ventana

1. **Cálculos sin Agrupamiento Completo:**
   - Las funciones de ventana permiten realizar operaciones como promedios, sumas, y rankings dentro de una partición de datos sin agrupar completamente los datos.
   
2. **Contexto Manteniendo el Orden:**
   - Mantienen el contexto de cada fila dentro de una partición, permitiendo realizar cálculos acumulativos y de comparación.
   
3. **Ventanas Definidas Dinámicamente:**
   - Las ventanas pueden ser definidas basándose en particiones y órdenes específicos, lo que permite una gran flexibilidad en el análisis de datos.

### Tipos Comunes de Funciones de Ventana

- **Agregaciones:**
  - `avg()`, `sum()`, `min()`, `max()`: Calculan el promedio, suma, mínimo y máximo dentro de la ventana.
  
- **Funciones de Rango y Ranking:**
  - `row_number()`, `rank()`, `dense_rank()`: Asignan números de fila y rangos dentro de la ventana.

- **Funciones de Desplazamiento:**
  - `lag()`, `lead()`: Permiten acceder a las filas anteriores o siguientes dentro de la ventana.

### Ejemplo de Uso

Considere un DataFrame de calificaciones de películas, donde queremos calcular el promedio de calificaciones por usuario, el ranking de calificaciones por género y año, y el cambio de calificación respecto a la calificación anterior dada por el mismo usuario. Las funciones de ventana nos permiten realizar estos cálculos de manera eficiente y mantener el contexto de cada calificación individual.

Con las funciones de ventana, podemos transformar y analizar los datos de manera avanzada, obteniendo insights que serían difíciles de lograr con simples agregaciones o transformaciones básicas.



### Calcular el Promedio de Calificaciones por Usuario

En este ejemplo, calcularemos el promedio de calificaciones de cada usuario para todas las películas que han calificado. Usaremos la función de ventana avg para realizar este cálculo.



In [25]:
from pyspark.sql.window import Window

# Suponiendo que movie_ratingsDF ya está cargado
windowSpec = Window.partitionBy("userId")

# Calcular el promedio de calificaciones por usuario
average_ratingsDF = movie_ratingsDF\
                    .withColumn(\
                                "average_rating", \
                                avg("rating")\
                                .over(windowSpec))

average_ratingsDF.show()


+-------+------+------+--------+--------------------+----+--------------------+--------------+
|movieId|userId|rating|    date|              genres|year|               title|average_rating|
+-------+------+------+--------+--------------------+----+--------------------+--------------+
|     39|    12|   4.0|20090710|   [Comedy, Romance]|1995|            Clueless|       4.39063|
|    168|    12|   5.0|20090710|[Action, Drama, R...|1995|        First Knight|       4.39063|
|    222|    12|   5.0|20090710|    [Drama, Romance]|1995|   Circle of Friends|       4.39063|
|    256|    12|   5.0|20090710|    [Comedy, Sci-Fi]|1994|              Junior|       4.39063|
|    261|    12|   4.5|20090710|             [Drama]|1994|        Little Women|       4.39063|
|    277|    12|   3.0|20090710|             [Drama]|1994|Miracle on 34th S...|       4.39063|
|    357|    12|   3.5|20090710|   [Comedy, Romance]|1994|Four Weddings and...|       4.39063|
|    543|    12|   3.5|20090710|[Comedy, Romance,.

### Calcular el Ranking de Calificaciones por Género y Año

En este ejemplo, calcularemos el ranking de las calificaciones de las películas dentro de cada género y año. Utilizaremos la función de ventana rank para determinar la posición de cada película en términos de calificación.



In [26]:
from pyspark.sql.functions import rank

explodedDF = movie_ratingsDF.withColumn("genre", explode("genres"))

windowSpec = Window.partitionBy("genre", "year").orderBy(col("rating").desc())

ranked_ratingsDF = explodedDF.withColumn("rank", rank().over(windowSpec))

ranked_ratingsDF.show()


+-------+------+------+--------+--------------------+----+--------------------+------------------+----+
|movieId|userId|rating|    date|              genres|year|               title|             genre|rank|
+-------+------+------+--------+--------------------+----+--------------------+------------------+----+
| 155589|    89|   3.0|20180307|[(no genres listed)]|1968|     Noin 7 veljestä|(no genres listed)|   1|
|   4800|    91|   2.5|20050405|[Action, Adventur...|1937|King Solomon's Mines|            Action|   1|
|   8481|   474|   3.0|20040602|[Action, Adventur...|1940|   Northwest Passage|            Action|   1|
|   8677|    68|   2.0|20060917|    [Action, Sci-Fi]|1940|Flash Gordon Conq...|            Action|   2|
|   8677|    88|   0.5|20120310|    [Action, Sci-Fi]|1940|Flash Gordon Conq...|            Action|   3|
|   1254|   290|   5.0|20001123|[Action, Adventur...|1948|Treasure of the S...|            Action|   1|
|   1254|   603|   5.0|20000709|[Action, Adventur...|1948|Treasu

### Calcular el Cambio de Calificación respecto a la Calificación Anterior por Usuario

En este ejemplo, calcularemos el cambio de calificación de una película respecto a la calificación anterior dada por el mismo usuario. Usaremos la función de ventana lag para acceder a la calificación anterior y calcular la diferencia.

In [28]:
from pyspark.sql.functions import lag


# Suponiendo que movie_ratingsDF ya está cargado
windowSpec = Window.partitionBy("userId").orderBy("date")

# Calcular el cambio de calificación respecto a la calificación anterior por usuario
change_ratingsDF = movie_ratingsDF.withColumn("previous_rating", lag("rating", 1).over(windowSpec)) \
                                  .withColumn("rating_change", col("rating") - col("previous_rating"))

change_ratingsDF.show()


+-------+------+------+--------+--------------------+----+--------------------+---------------+-------------+
|movieId|userId|rating|    date|              genres|year|               title|previous_rating|rating_change|
+-------+------+------+--------+--------------------+----+--------------------+---------------+-------------+
|     39|    12|   4.0|20090710|   [Comedy, Romance]|1995|            Clueless|           NULL|         NULL|
|    168|    12|   5.0|20090710|[Action, Drama, R...|1995|        First Knight|            4.0|          1.0|
|    222|    12|   5.0|20090710|    [Drama, Romance]|1995|   Circle of Friends|            5.0|          0.0|
|    256|    12|   5.0|20090710|    [Comedy, Sci-Fi]|1994|              Junior|            5.0|          0.0|
|    261|    12|   4.5|20090710|             [Drama]|1994|        Little Women|            5.0|         -0.5|
|    277|    12|   3.0|20090710|             [Drama]|1994|Miracle on 34th S...|            4.5|         -1.5|
|    357| 

In [6]:
# Alternativa: metodo count
movie_ratingsDF_grouped_by_rating.count().show()

+------+-----+
|rating|count|
+------+-----+
|   5.0|13211|
|   2.5| 5550|
|   0.5| 1370|
|   1.0| 2811|
|   3.5|13136|
|   1.5| 1791|
|   3.0|20047|
|   2.0| 7551|
|   4.0|26818|
|   4.5| 8551|
+------+-----+



Ordenamos la lista para que se vea mejor:

In [13]:
movie_ratingsDF_grouped_by_rating.count().orderBy(desc("rating")).show()

+------+-----+
|rating|count|
+------+-----+
|   5.0|13211|
|   4.5| 8551|
|   4.0|26818|
|   3.5|13136|
|   3.0|20047|
|   2.5| 5550|
|   2.0| 7551|
|   1.5| 1791|
|   1.0| 2811|
|   0.5| 1370|
+------+-----+



## ¿Cuál es la película con la mayor cantidad de reseñas? 

In [33]:
movie_ratingsDF_grouped_by_movieID = movie_ratingsDF.groupBy("title").count().orderBy(desc("count"))
movie_ratingsDF_grouped_by_movieID.show(1)


+------------+-----+
|       title|count|
+------------+-----+
|Forrest Gump|  329|
+------------+-----+
only showing top 1 row



In [22]:
movie_ratingsDF.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- userId: integer (nullable = true)
 |-- rating: decimal(2,1) (nullable = true)
 |-- date: string (nullable = true)
 |-- genres: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- year: integer (nullable = true)
 |-- title: string (nullable = true)



In [24]:
movie_ratingsDF.show(2)

+-------+------+------+--------+--------------------+----+----------------+
|movieId|userId|rating|    date|              genres|year|           title|
+-------+------+------+--------+--------------------+----+----------------+
|      1|     1|   4.0|20000730|[Adventure, Anima...|1995|       Toy Story|
|      3|     1|   4.0|20000730|   [Comedy, Romance]|1995|Grumpier Old Men|
+-------+------+------+--------+--------------------+----+----------------+
only showing top 2 rows

